In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# data can be found here: https://www.kaggle.com/ashishpatel26/facial-expression-recognitionferchallenge
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf

from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.losses import CategoricalCrossentropy

df = pd.read_csv('../input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv')

In [ ]:
df.head()

In [ ]:
images = []
for i in range(df.shape[0]):
    # the pixels column is a string of space-separated numbers, convert this to a list where each entry is separated by a space
    image = df['pixels'][i].split()
    # convert from strings to floats, alter to be in range 0-1 instead of 0-255
    image = [float(i)/255 for i in image]
    # reshape to be 48 x 48
    image = np.reshape(image, (48, 48))
    # expand dims to have channels, so the array is shape 48x48x1
    image = np.expand_dims(image, axis=2)
    images.append(image)

In [ ]:
images = np.stack(images, axis=0)

In [ ]:
# convert targets to series
targets = df['emotion']

In [ ]:
label_to_hr = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

In [ ]:
fig = plt.figure(figsize=(10,10))
for i in range(10):
    fig.add_subplot(5, 5, i+1)
    plt.imshow(images[i], cmap='gray')
    label = label_to_hr[targets[i]]
    plt.title(label)
    plt.axis('off')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
(X_train, X_valid, y_train, y_valid) = train_test_split(images, targets, test_size=0.2, random_state=999)

In [ ]:
X_train.shape

In [ ]:
type(X_train)

In [ ]:
images.shape

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(y_valid)

y_train_bin = lb.transform(y_train)
y_valid_bin = lb.transform(y_valid)

In [ ]:
lb.classes_

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    shear_range = 0.2,
    zoom_range = 0.2,
    rotation_range = 20,
    horizontal_flip = True
)

valid_datagen = ImageDataGenerator()

In [ ]:
batch_size=10

train_generator = train_datagen.flow(X_train, y_train_bin, batch_size=batch_size)
valid_generator = valid_datagen.flow(X_valid, y_valid_bin, batch_size=batch_size)

In [ ]:
def build_network():
    input_layer = Input(shape=(48, 48, 1))
    x = Conv2D(filters=32,
               kernel_size=(3, 3),
               padding='same',
               strides=(1, 1))(input_layer)
    x = ReLU()(x)
    x = Dropout(rate=0.5)(x)

    x = Flatten()(x)
    x = Dense(units=7)(x)
    output = Softmax()(x)

    model = Model(inputs=input_layer, outputs=output)
    return model

In [ ]:
model = build_network()

In [ ]:
epochs = 50
model.compile(loss=CategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [ ]:
for iteration in train_generator:
    print(iteration)
    break

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              min_delta=0, 
                                              patience=5, 
                                              verbose=0,
                                              mode='auto', 
                                              baseline=None, 
                                              restore_best_weights=False
                                             )

In [ ]:
epochs = 50
history = model.fit(train_generator, validation_data=valid_generator, epochs=epochs, callbacks=[early_stop])

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from tensorflow.keras.applications import ResNet50V2

n_layers = 150
img_shape = (48, 48, 3)

In [ ]:
base_model  = ResNet50V2(input_shape=img_shape, include_top=False, weights='imagenet')
head_model = base_model
for layers in base_model.layers[:n_layers]:
    layers.trainable = False
head_model = head_model.output
head_model = tf.keras.layers.GlobalMaxPooling2D()(head_model)
head_model = tf.keras.layers.Flatten(name="Flatten")(head_model)
head_model = tf.keras.layers.Dense(1024, activation='relu')(head_model)
head_model = tf.keras.layers.Dropout(0.2)(head_model)
prediction_layer = tf.keras.layers.Dense(7, activation='softmax')(head_model)
model = tf.keras.Model(inputs=base_model.input, outputs=prediction_layer)